In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg16 import VGG16 ,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
train_path='train'
test_path='test'

In [6]:
vgg= VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [7]:
for layer in vgg.layers:
    layer.trainable=False

In [55]:
no_of_folder=glob('train/*')
print(len(no_of_folder))

2


In [9]:
x=Flatten()(vgg.output)
prediction=Dense(len(no_of_folder),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
test_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
train_set=train_datagen.flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical')

Found 2637 images belonging to 2 classes.


In [14]:
test_set=test_datagen.flow_from_directory(
    test_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical')

Found 660 images belonging to 2 classes.


In [15]:
model_history=model.fit(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
)



Epoch 1/10


c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 68s 14s/step - accuracy: 0.5439 - loss: 3.0290 - val_accuracy: 0.7606 - val_loss: 1.9413
Epoch 2/10


c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


5/5 ━━━━━━━━━━━━━━━━━━━━ 65s 16s/step - accuracy: 0.7497 - loss: 1.9316 - val_accuracy: 0.7879 - val_loss: 1.5045
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 65s 15s/step - accuracy: 0.8141 - loss: 1.9896 - val_accuracy: 0.7879 - val_loss: 2.0612
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 59s 14s/step - accuracy: 0.7825 - loss: 1.9158 - val_accuracy: 0.7606 - val_loss: 2.2181
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 59s 14s/step - accuracy: 0.8047 - loss: 2.0784 - val_accuracy: 0.7970 - val_loss: 2.0321
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 63s 15s/step - accuracy: 0.8309 - loss: 1.7517 - val_accuracy: 0.8030 - val_loss: 2.2394
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 91s 21s/step - accuracy: 0.8001 - loss: 1.6931 - val_accuracy: 0.7848 - val_loss: 1.8477
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 84s 20s/step - accuracy: 0.8002 - loss: 1.8347 - val_accuracy: 0.8076 - val_loss: 2.0591
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 66s 16s/step - accuracy: 0.8593 - loss: 1.3278 - val_accuracy: 0.8000 - val_loss: 1.8909
Epoch 10/10

In [16]:
loss, accuracy = model.evaluate(test_set)
print('Test Accuracy: {:.2f}%'.format(accuracy * 100))
print('Test Loss: {:.4f}'.format(loss))

21/21 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.8207 - loss: 2.0270
Test Accuracy: 81.82%
Test Loss: 2.1210


In [18]:
model.save('my_model.h5')

In [19]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [20]:
model = load_model('my_model.h5')

In [21]:
def get_class_name(idx):
    classes = sorted(os.listdir(train_path))
    return classes[idx]

In [68]:
i = 'mal.jpg'
img = image.load_img(i, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [69]:
predictions = model.predict(x)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


In [70]:
predicted_class_index = np.argmax(predictions)
predicted_class_name = get_class_name(predicted_class_index)

In [71]:
print(f'Predicted class index: {predicted_class_index}')
print(f'Predicted class name: {predicted_class_name}')

Predicted class index: 1
Predicted class name: malignant
